In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an instance of the OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(api_key=api_key, model="gpt-4o")

## Multi Model Chat

In [8]:
from io import BytesIO

import requests
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image

assistant_1 = AssistantAgent(name='assistant',
                           model_client=model_client,
                           description="Give output in JSON",
                           system_message="You are a helpful assistant that provides accurate information about history.Give output in JSON"
                           )

pil_image = Image.open(BytesIO(requests.get("https://picsum.photos/id/9/200/400").content))
img = AGImage(pil_image)

multi_model_message = MultiModalMessage(
    content = ['What is there in the image?',img],
    source="User"
)

result = await assistant_1.run(task=multi_model_message)
result.messages[-1].content

'```json\n{\n  "description": "The image shows a laptop with a silver body on a wooden surface. The screen is blank, and the keyboard features black keys."\n}\n```'

In [10]:
## Previous message history is also stored in the Agent memory

result = await assistant_1.run(task="What I asked you in the last question?")
result.messages[-1].content

'```json\n{\n  "question": "What is there in the image?"\n}\n```'

## Agent With Tool

In [11]:
def get_weather(city:str)->str:
    # Simulate a weather API call
    return f"The weather in {city} is sunny with a temperature of 25°C."


weather_agent = AssistantAgent(
    name='weather_agent',
    model_client=model_client,
    description="You are a weather assistant that provides accurate weather information.",
    system_message="You are a helpful assistant that provides accurate weather information by using the get_weather.",
    tools=[get_weather]
)

result = await weather_agent.run(task="What is the weather in Delhi?")
result.messages[-1].content


'The weather in Delhi is sunny with a temperature of 25°C.'